In [1]:
import bs4 as bs
import urllib.request
import time
import pandas as pd
import json
import os
import time

from datetime import datetime

In [2]:
class ImmoScout24Scraper():
    def __init__(self, url, folder_path, num_pages = 1):
        """
            url - e.g. 'https://www.immobilienscout24.de/Suche/S-/P-?/Haus-Kauf'
                        '?'' will be replaced by current page number
        """  
        self.url_parts = url.split('?')
        if num_pages == 'all':
            url = self.url_parts[0]+"1"+self.url_parts[1]
            print(url)
            soup = self.create_soup(url)
            self.num_pages = len(soup.select('select[aria-label=Seitenauswahl]')[0].find_all('option'))
            print("Number of pages: ", self.num_pages)
        else:
            self.num_pages = num_pages
            
        self.all_objects = []
        self.folder_path = folder_path
        self.main()

    def main(self):
        print("START!")
        try:
            for page_num in range(self.num_pages):
                page_num = page_num + 1 
                url = self.url_parts[0] + str(page_num) + self.url_parts[1]
                print('Page: ', page_num, ' | Current url: ' + url)
                soup = self.create_soup(url)  
                # get page objects from object listing page 
                page_objects = self.extract_page_objects(soup)
                # get single exposes
                exposes = self.process_page_objects(page_objects)
                # save all exposes from one object listing page
                self.save(exposes, page_num)
        except Exception as e: 
            print(str(datetime.now())+": " + str(e))
        print("FINISH!")
        
    def create_soup(self, url):
        data = urllib.request.urlopen(url)
        data = data.read()
        soup = bs.BeautifulSoup(data,'lxml')
        return soup
    
    def extract_page_objects(self, soup):
        page_objects = []
        for result in soup.find_all('li', {'class': 'result-list__listing'}):
            page_objects.append('/expose/' + str(result['data-id']))
            
        # get unique objects
        page_objects = list(set(page_objects))
        return page_objects
            
    def process_page_objects(self, page_objects):
        exposes = pd.DataFrame()
        for item in page_objects:
            if item not in self.all_objects:
                # print(item)
                soup_single_item = bs.BeautifulSoup(urllib.request.urlopen('https://www.immobilienscout24.de'+item).read(),'lxml')
                data = pd.DataFrame(json.loads(str(soup_single_item.find_all("script")).split("keyValues = ")[1].split("}")[0]+str("}")),index=[str(datetime.now())])
                data["URL"] = str(item)

                description = []

                for i in soup_single_item.find_all("pre"):
                    description.append(i.text)
                data["beschreibung"] = str(description)
                
                exposes = exposes.append(data)
                
        if self.all_objects:
            self.all_objects = list(set(self.all_objects + page_objects))
        else:
            self.all_objects = page_objects
            
        return exposes
    
    def save(self, exposes, page_num):
        file_path = self.folder_path+str(datetime.now())[:10]+"-Haus-Kauf_P-" + str(page_num) + ".csv"
        exposes.to_csv(file_path, sep=";", decimal=",", encoding = "utf-8") 
        

In [3]:
current_path = os.getcwd()   
folder_path = current_path + '/data_raw_09082019/'
url = 'https://www.immobilienscout24.de/Suche/S-/P-?/Haus-Kauf'

scraper = ImmoScout24Scraper(url, folder_path, num_pages = 'all')

https://www.immobilienscout24.de/Suche/S-/P-1/Haus-Kauf
Number of pages:  2887
START!
Page:  1  | Current url: https://www.immobilienscout24.de/Suche/S-/P-1/Haus-Kauf


/Users/matthies/anaconda3/envs/ai/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Page:  2  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2/Haus-Kauf
Page:  3  | Current url: https://www.immobilienscout24.de/Suche/S-/P-3/Haus-Kauf
Page:  4  | Current url: https://www.immobilienscout24.de/Suche/S-/P-4/Haus-Kauf
Page:  5  | Current url: https://www.immobilienscout24.de/Suche/S-/P-5/Haus-Kauf
Page:  6  | Current url: https://www.immobilienscout24.de/Suche/S-/P-6/Haus-Kauf
Page:  7  | Current url: https://www.immobilienscout24.de/Suche/S-/P-7/Haus-Kauf
Page:  8  | Current url: https://www.immobilienscout24.de/Suche/S-/P-8/Haus-Kauf
Page:  9  | Current url: https://www.immobilienscout24.de/Suche/S-/P-9/Haus-Kauf
Page:  10  | Current url: https://www.immobilienscout24.de/Suche/S-/P-10/Haus-Kauf
Page:  11  | Current url: https://www.immobilienscout24.de/Suche/S-/P-11/Haus-Kauf
Page:  12  | Current url: https://www.immobilienscout24.de/Suche/S-/P-12/Haus-Kauf
Page:  13  | Current url: https://www.immobilienscout24.de/Suche/S-/P-13/Haus-Kauf
Page:  14  | Current

Page:  101  | Current url: https://www.immobilienscout24.de/Suche/S-/P-101/Haus-Kauf
Page:  102  | Current url: https://www.immobilienscout24.de/Suche/S-/P-102/Haus-Kauf
Page:  103  | Current url: https://www.immobilienscout24.de/Suche/S-/P-103/Haus-Kauf
Page:  104  | Current url: https://www.immobilienscout24.de/Suche/S-/P-104/Haus-Kauf
Page:  105  | Current url: https://www.immobilienscout24.de/Suche/S-/P-105/Haus-Kauf
Page:  106  | Current url: https://www.immobilienscout24.de/Suche/S-/P-106/Haus-Kauf
Page:  107  | Current url: https://www.immobilienscout24.de/Suche/S-/P-107/Haus-Kauf
Page:  108  | Current url: https://www.immobilienscout24.de/Suche/S-/P-108/Haus-Kauf
Page:  109  | Current url: https://www.immobilienscout24.de/Suche/S-/P-109/Haus-Kauf
Page:  110  | Current url: https://www.immobilienscout24.de/Suche/S-/P-110/Haus-Kauf
Page:  111  | Current url: https://www.immobilienscout24.de/Suche/S-/P-111/Haus-Kauf
Page:  112  | Current url: https://www.immobilienscout24.de/Suche

Page:  198  | Current url: https://www.immobilienscout24.de/Suche/S-/P-198/Haus-Kauf
Page:  199  | Current url: https://www.immobilienscout24.de/Suche/S-/P-199/Haus-Kauf
Page:  200  | Current url: https://www.immobilienscout24.de/Suche/S-/P-200/Haus-Kauf
Page:  201  | Current url: https://www.immobilienscout24.de/Suche/S-/P-201/Haus-Kauf
Page:  202  | Current url: https://www.immobilienscout24.de/Suche/S-/P-202/Haus-Kauf
Page:  203  | Current url: https://www.immobilienscout24.de/Suche/S-/P-203/Haus-Kauf
Page:  204  | Current url: https://www.immobilienscout24.de/Suche/S-/P-204/Haus-Kauf
Page:  205  | Current url: https://www.immobilienscout24.de/Suche/S-/P-205/Haus-Kauf
Page:  206  | Current url: https://www.immobilienscout24.de/Suche/S-/P-206/Haus-Kauf
Page:  207  | Current url: https://www.immobilienscout24.de/Suche/S-/P-207/Haus-Kauf
Page:  208  | Current url: https://www.immobilienscout24.de/Suche/S-/P-208/Haus-Kauf
Page:  209  | Current url: https://www.immobilienscout24.de/Suche

...

Page:  2304  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2304/Haus-Kauf
Page:  2305  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2305/Haus-Kauf
Page:  2306  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2306/Haus-Kauf
Page:  2307  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2307/Haus-Kauf
Page:  2308  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2308/Haus-Kauf
Page:  2309  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2309/Haus-Kauf
Page:  2310  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2310/Haus-Kauf
Page:  2311  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2311/Haus-Kauf
Page:  2312  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2312/Haus-Kauf
Page:  2313  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2313/Haus-Kauf
Page:  2314  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2314/Haus-Kauf
Page:  2315  | Current url: https://www.imm

Page:  2399  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2399/Haus-Kauf
Page:  2400  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2400/Haus-Kauf
Page:  2401  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2401/Haus-Kauf
Page:  2402  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2402/Haus-Kauf
Page:  2403  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2403/Haus-Kauf
Page:  2404  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2404/Haus-Kauf
Page:  2405  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2405/Haus-Kauf
Page:  2406  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2406/Haus-Kauf
Page:  2407  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2407/Haus-Kauf
Page:  2408  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2408/Haus-Kauf
Page:  2409  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2409/Haus-Kauf
Page:  2410  | Current url: https://www.imm

Page:  2494  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2494/Haus-Kauf
Page:  2495  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2495/Haus-Kauf
Page:  2496  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2496/Haus-Kauf
Page:  2497  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2497/Haus-Kauf
Page:  2498  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2498/Haus-Kauf
Page:  2499  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2499/Haus-Kauf
Page:  2500  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2500/Haus-Kauf
Page:  2501  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2501/Haus-Kauf
Page:  2502  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2502/Haus-Kauf
Page:  2503  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2503/Haus-Kauf
Page:  2504  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2504/Haus-Kauf
Page:  2505  | Current url: https://www.imm

Page:  2589  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2589/Haus-Kauf
Page:  2590  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2590/Haus-Kauf
Page:  2591  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2591/Haus-Kauf
Page:  2592  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2592/Haus-Kauf
Page:  2593  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2593/Haus-Kauf
Page:  2594  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2594/Haus-Kauf
Page:  2595  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2595/Haus-Kauf
Page:  2596  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2596/Haus-Kauf
Page:  2597  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2597/Haus-Kauf
Page:  2598  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2598/Haus-Kauf
Page:  2599  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2599/Haus-Kauf
Page:  2600  | Current url: https://www.imm

Page:  2684  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2684/Haus-Kauf
Page:  2685  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2685/Haus-Kauf
Page:  2686  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2686/Haus-Kauf
Page:  2687  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2687/Haus-Kauf
Page:  2688  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2688/Haus-Kauf
Page:  2689  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2689/Haus-Kauf
Page:  2690  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2690/Haus-Kauf
Page:  2691  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2691/Haus-Kauf
Page:  2692  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2692/Haus-Kauf
Page:  2693  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2693/Haus-Kauf
Page:  2694  | Current url: https://www.immobilienscout24.de/Suche/S-/P-2694/Haus-Kauf
Page:  2695  | Current url: https://www.imm